# Read old dataset

In [2]:
import pandas as pd

df=pd.read_csv('final_creek_and_weather_dataset.csv')
df.head(10)
# Convert the date column to datetime if not already
df['datetime'] = pd.to_datetime(df['datetime'], errors='coerce')

# Drop rows with invalid dates just in case
df = df.dropna(subset=['datetime'])

# Print the range
print("Date range:")
print(f"Start: {df['datetime'].min()}")
print(f"End:   {df['datetime'].max()}")

Date range:
Start: 2021-04-30 21:50:00
End:   2025-09-04 00:00:00


In [5]:
import pandas as pd

def load_lbnl_weather(csv_path: str) -> pd.DataFrame:
    """
    Load and clean LBNL weather data file.
    Removes the units row and parses timestamps.
    """
    # Read with correct comma separator
    df = pd.read_csv(csv_path, sep=",", skiprows=[1])

    # Strip whitespace / invisible chars just in case
    df.columns = (
        df.columns
        .str.replace(r"\s+", "", regex=True)
        .str.replace("\xa0", "", regex=False)
        .str.strip()
    )

    # Convert to datetime
    df["Date_Time"] = pd.to_datetime(df["Date_Time"], errors="coerce", utc=True)

    # Sort chronologically
    df = df.sort_values("Date_Time").reset_index(drop=True)

    return df


# Example usage:
lbnl = load_lbnl_weather("lbnl data/LBNL1.2025-10-19.csv")

In [6]:
import requests
import pandas as pd
from datetime import datetime, timedelta

# -----------------------------
# CONFIGURATION
# -----------------------------
LAT = 37.8715   # Example: Berkeley, CA
LON = -122.2730
START_DATE = "2025-09-04"
END_DATE = "2025-10-20"
OUTPUT_CSV = "open_metero_10-19-rain_data.csv"

# -----------------------------
# FUNCTION TO FETCH ONE MONTH OF DATA
# -----------------------------
def fetch_rain_data(lat, lon, start_date, end_date):
    url = "https://archive-api.open-meteo.com/v1/archive"
    params = {
        "latitude": lat,
        "longitude": lon,
        "start_date": start_date,
        "end_date": end_date,
        "hourly": ["precipitation", "rain", "showers"],
        "timezone": "auto",
    }
    response = requests.get(url, params=params)
    response.raise_for_status()
    data = response.json()

    if "hourly" not in data:
        print(f"No data for {start_date} to {end_date}")
        return pd.DataFrame()

    df = pd.DataFrame(data["hourly"])
    df["time"] = pd.to_datetime(df["time"])
    return df

# -----------------------------
# MAIN SCRIPT
# -----------------------------
def main():
    start = datetime.fromisoformat(START_DATE)
    end = datetime.fromisoformat(END_DATE)
    all_data = []

    while start < end:
        month_end = (start + timedelta(days=32)).replace(day=1) - timedelta(days=1)
        if month_end > end:
            month_end = end

        print(f"Fetching {start.date()} → {month_end.date()}")
        df = fetch_rain_data(LAT, LON, start.date(), month_end.date())
        if not df.empty:
            all_data.append(df)

        # Move to the next month
        start = month_end + timedelta(days=1)

    # Combine all months
    if all_data:
        full_df = pd.concat(all_data, ignore_index=True)
        full_df.to_csv(OUTPUT_CSV, index=False)
        print(f"✅ Saved full dataset to {OUTPUT_CSV}")
    else:
        print("⚠️ No data fetched.")

if __name__ == "__main__":
    main()


Fetching 2025-09-04 → 2025-09-30
Fetching 2025-10-01 → 2025-10-20
✅ Saved full dataset to open_metero_10-19-rain_data.csv


In [7]:
import requests
import pandas as pd
from datetime import datetime
import os

def fetch_all_creek_data(save_csv=True, output_dir="./data"):
    """
    Fetches all Strawberry Creek site data, combines into one DataFrame,
    and optionally saves it as a single CSV file.
    """
    BASE_URL = "https://www.strawberrycreek.org/api/creek-data/"
    TOKEN = "8decd09a7a4035a4d3e012191fe41d6db39f9c1e"
    SITES = [
        {"table": "north_fork_0", "label": "North Fork 0"},
        {"table": "scnf010", "label": "North Fork 1 (Wickson Footbridge)"},
        {"table": "south_fork_0", "label": "South Fork 0"},
        {"table": "south_fork_1", "label": "South Fork 1"},
        {"table": "south_fork_2", "label": "South Fork 2"},
        {"table": "south_fork_3", "label": "South Fork 3"},
        {"table": "university_house", "label": "University House"},
        {"table": "oxford", "label": "Oxford Street"},
        {"table": "codornices", "label": "Codornices Creek"},
    ]
    VARIABLES = [
        "Meter_Hydros21_Cond",
        "Meter_Hydros21_Depth",
        "Meter_Hydros21_Temp",
        "EnviroDIY_Mayfly_Batt",
    ]

    start_date = "2025-09-03"
    end_date = datetime.now().strftime("%Y-%m-%d")
    vars_string = ",".join(VARIABLES)
    headers = {"Authorization": f"Token {TOKEN}"}

    all_dataframes = []
    print("🌊 Starting creek data fetch for all sites...")

    for site in SITES:
        params = {"site": site["table"], "start": start_date, "end": end_date, "vars": vars_string}
        try:
            print(f"Fetching data for: {site['label']}...")
            response = requests.get(BASE_URL, headers=headers, params=params)
            response.raise_for_status()
            data = response.json()

            if data:
                df = pd.DataFrame(data)
                df["location"] = site["table"]
                df["site_label"] = site["label"]
                all_dataframes.append(df)
                print(f"✅ Success! Found {len(df)} rows for {site['label']}.")
            else:
                print(f"⚠️ No data for {site['label']}.")
        except requests.exceptions.RequestException as e:
            print(f"❌ Failed for {site['label']}. Error: {e}")

    if not all_dataframes:
        print("\n⚠️ No data was fetched.")
        return None

    # Combine all site data into one DataFrame
    final_df = pd.concat(all_dataframes, ignore_index=True)

    # Ensure output directory exists
    if save_csv:
        os.makedirs(output_dir, exist_ok=True)
        output_path = os.path.join(output_dir, f"creek_data_all_sites_{end_date}.csv")
        final_df.to_csv(output_path, index=False)
        print(f"💾 Saved combined dataset to: {output_path}")

    return final_df
fetch_all_creek_data(save_csv=True, output_dir="creek-data-10-19")

🌊 Starting creek data fetch for all sites...
Fetching data for: North Fork 0...
✅ Success! Found 5003 rows for North Fork 0.
Fetching data for: North Fork 1 (Wickson Footbridge)...
✅ Success! Found 5436 rows for North Fork 1 (Wickson Footbridge).
Fetching data for: South Fork 0...
✅ Success! Found 5459 rows for South Fork 0.
Fetching data for: South Fork 1...
✅ Success! Found 5459 rows for South Fork 1.
Fetching data for: South Fork 2...
✅ Success! Found 5461 rows for South Fork 2.
Fetching data for: South Fork 3...
✅ Success! Found 10 rows for South Fork 3.
Fetching data for: University House...
✅ Success! Found 5454 rows for University House.
Fetching data for: Oxford Street...
✅ Success! Found 5454 rows for Oxford Street.
Fetching data for: Codornices Creek...
✅ Success! Found 5473 rows for Codornices Creek.
💾 Saved combined dataset to: creek-data-10-19\creek_data_all_sites_2025-10-30.csv


C:\Users\andre\AppData\Local\Temp\ipykernel_22280\2236127852.py:63: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df = pd.concat(all_dataframes, ignore_index=True)


,DateTimeUTC,Meter_Hydros21_Cond,Meter_Hydros21_Depth,Meter_Hydros21_Temp,EnviroDIY_Mayfly_Batt,location,site_label
0,2025-09-03 00:00:00,445.3,203.700,18.20,4.397,north_fork_0,North Fork 0
1,2025-09-03 00:15:00,442.0,203.200,18.20,4.397,north_fork_0,North Fork 0
2,2025-09-03 00:30:00,441.2,202.700,18.20,4.397,north_fork_0,North Fork 0
3,2025-09-03 01:00:00,443.5,203.300,18.20,4.397,north_fork_0,North Fork 0
4,2025-09-03 01:15:00,443.7,203.700,18.20,4.397,north_fork_0,North Fork 0
...,...,...,...,...,...,...,...
43204,2025-10-29 23:00:00,279.5,286.512,14.94,NaN,codornices,Codornices Creek
43205,2025-10-29 23:15:00,279.7,286.512,14.90,NaN,codornices,Codornices Creek
43206,2025-10-29 23:30:00,279.8,286.512,15.02,NaN,codornices,Codornices Creek
43207,2025-10-29 23:45:00,280.1,286.512,15.06,NaN,codornices,Codornices Creek


In [34]:
import pandas as pd

# === FILE PATHS ===
creek_file = "creek-data-10-19/creek_data_all_sites_2025-10-19.csv"
lbnl_file = "lbnl data/LBNL1.2025-10-19.csv"
openmeteo_file = "open_metero_10-19-rain_data.csv"
output_file = "merged_creek_weather.csv"

# === LOAD DATA ===
creek = pd.read_csv(creek_file)
openmeteo = pd.read_csv(openmeteo_file)

# Detect if LBNL has a units row and skip it automatically
with open(lbnl_file) as f:
    first_line = f.readline()
    second_line = f.readline()
skip_rows = [1] if any(u in second_line for u in ["Celsius", "Millimeters", "Pascals", "%"]) else []

lbnl = pd.read_csv(lbnl_file, skiprows=skip_rows)

# === PARSE DATETIMES ===
creek["DateTimeUTC"] = pd.to_datetime(creek["DateTimeUTC"], utc=True, errors="coerce")
lbnl["Date_Time"] = pd.to_datetime(lbnl["Date_Time"], utc=True, errors="coerce")
openmeteo["time"] = pd.to_datetime(openmeteo["time"], utc=True, errors="coerce")

# === PREP OPEN-METEO RAIN ===
openmeteo = openmeteo[["time", "rain"]].rename(columns={"time": "DateTimeUTC", "rain": "rain_openmeteo_mm"})
openmeteo = openmeteo.set_index("DateTimeUTC")
openmeteo = openmeteo.resample("15min").interpolate(method="linear")
openmeteo["rain_openmeteo_mm"] /= 4  # convert mm/hour to mm/15min

# === PREP LBNL WEATHER ===
lbnl = lbnl.rename(columns={"Date_Time": "DateTimeUTC"})
lbnl = lbnl.set_index("DateTimeUTC")

# Clean rain column
if "precip_accum_fifteen_minute_set_1" in lbnl.columns:
    lbnl["rain_lbnl_mm"] = pd.to_numeric(lbnl["precip_accum_fifteen_minute_set_1"], errors="coerce")
else:
    lbnl["rain_lbnl_mm"] = 0.0

lbnl_15 = lbnl.resample("15min").mean(numeric_only=True)

# === MERGE ALL ===
creek = creek.set_index(pd.to_datetime(creek["DateTimeUTC"], utc=True))
merged = creek.join([lbnl_15, openmeteo], how="left")

# === FINAL RAIN ===
merged["rain_mm"] = merged[["rain_lbnl_mm", "rain_openmeteo_mm"]].max(axis=1)


# === CLEANUP ===
merged.reset_index(drop=True, inplace=True)
merged.to_csv(output_file, index=False)

print(f"✅ Merged dataset saved to: {output_file}")
print(f"Rows: {len(merged)}, Columns: {len(merged.columns)}")
print("Columns:", list(merged.columns))



AttributeError: 'DataFrame' object has no attribute 'to_datetime'

In [26]:
pd.read_csv('merged_creek_weather.csv')

,DateTimeUTC,Meter_Hydros21_Cond,Meter_Hydros21_Depth,Meter_Hydros21_Temp,EnviroDIY_Mayfly_Batt,location,site_label,pressure_set_1,air_temp_set_1,air_temp_set_2,...,volt_set_1,wind_chill_set_1d,heat_index_set_1d,dew_point_temperature_set_1d,altimeter_set_1d,sea_level_pressure_set_1d,wet_bulb_temp_set_1d,rain_lbnl_mm,rain_openmeteo_mm,rain_mm
0,2025-09-03 00:00:00+00:00,445.3,203.700,18.20,4.397,north_fork_0,North Fork 0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2025-09-03 00:15:00+00:00,442.0,203.200,18.20,4.397,north_fork_0,North Fork 0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2025-09-03 00:30:00+00:00,441.2,202.700,18.20,4.397,north_fork_0,North Fork 0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2025-09-03 01:00:00+00:00,443.5,203.300,18.20,4.397,north_fork_0,North Fork 0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2025-09-03 01:15:00+00:00,443.7,203.700,18.20,4.397,north_fork_0,North Fork 0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35013,2025-10-18 23:00:00+00:00,287.9,286.512,15.12,NaN,codornices,Codornices Creek,98320.0,23.83,23.32,...,13.31,NaN,NaN,11.11,101538.08,101419.88,16.05,0.0,0.0,0.0
35014,2025-10-18 23:15:00+00:00,288.1,283.464,14.99,NaN,codornices,Codornices Creek,98323.0,23.75,23.43,...,13.33,NaN,NaN,11.01,101541.18,101423.80,15.97,0.0,0.0,0.0
35015,2025-10-18 23:30:00+00:00,288.2,286.512,15.11,NaN,codornices,Codornices Creek,98325.0,23.77,23.66,...,13.33,NaN,NaN,10.79,101543.24,101425.69,15.87,0.0,0.0,0.0
35016,2025-10-18 23:45:00+00:00,288.6,283.464,15.17,NaN,codornices,Codornices Creek,98336.0,23.84,23.70,...,13.35,NaN,NaN,10.59,101554.60,101436.23,15.80,0.0,0.0,0.0


In [27]:
# === Sanity check: compare LBNL vs Open-Meteo rain ===
rain_df = merged[["rain_lbnl_mm", "rain_openmeteo_mm"]].dropna()

# 1️⃣ Number of mismatches: one says 0, the other >0
mismatch_count = ((rain_df["rain_lbnl_mm"] == 0) & (rain_df["rain_openmeteo_mm"] > 0)) | \
                 ((rain_df["rain_lbnl_mm"] > 0) & (rain_df["rain_openmeteo_mm"] == 0))
mismatch_count = mismatch_count.sum()
print(f"Number of mismatches (one 0, other >0): {mismatch_count}")

# 2️⃣ Average absolute difference between readings
avg_diff = (rain_df["rain_lbnl_mm"] - rain_df["rain_openmeteo_mm"]).abs().mean()
print(f"Average absolute difference between LBNL and Open-Meteo rain (mm/15min): {avg_diff:.3f}")


Number of mismatches (one 0, other >0): 3913
Average absolute difference between LBNL and Open-Meteo rain (mm/15min): 0.022


In [28]:
print(len(rain_df))

34245


In [29]:
import pandas as pd

# === FILE PATHS ===
old_file = "final_creek_and_weather_dataset.csv"  # your old CSV
new_file = "merged_creek_weather.csv"  # the new merged data from sensors + LBNL + Open-Meteo
output_file = "weather-creek-10-19-25.csv"

# === LOAD DATA ===
old = pd.read_csv(old_file, parse_dates=["datetime"])
new = pd.read_csv(new_file, parse_dates=["DateTimeUTC"])

# === MAKE TIMEZONE CONSISTENT ===
# old CSV was tz-naive → convert to UTC
old["datetime"] = pd.to_datetime(old["datetime"], errors="coerce", utc=True)
new = new.rename(columns={"DateTimeUTC": "datetime"})
# new["datetime"] is already tz-aware from earlier steps

# === ALIGN COLUMNS ===
for col in old.columns:
    if col not in new.columns:
        new[col] = pd.NA
for col in new.columns:
    if col not in old.columns:
        old[col] = pd.NA

# === CONCAT AND SORT ===
combined = pd.concat([old, new], ignore_index=True, sort=False)
combined = combined.sort_values("datetime").reset_index(drop=True)

# === SAVE ===
combined.to_csv(output_file, index=False)
print(f"✅ Updated CSV saved: {output_file}")
print(f"Total rows: {len(combined)}, Total columns: {len(combined.columns)}")


C:\Users\andre\AppData\Local\Temp\ipykernel_22280\271854845.py:27: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined = pd.concat([old, new], ignore_index=True, sort=False)


✅ Updated CSV saved: weather-creek-10-19-25.csv
Total rows: 514558, Total columns: 62


In [30]:
df_new = pd.read_csv('weather-creek-10-19-25.csv')

C:\Users\andre\AppData\Local\Temp\ipykernel_22280\3098699673.py:1: DtypeWarning: Columns (43) have mixed types. Specify dtype option on import or set low_memory=False.
  df_new = pd.read_csv('weather-creek-10-19-25.csv')


In [31]:
df_new

,Unnamed: 0,Unnamed: 0_x,datetime,conductivity,depth,temperature,location,Unnamed: 0_y,BaroPres,Batt_Volt_Avg,...,precip_accum_fifteen_minute_set_1,volt_set_1,wind_chill_set_1d,heat_index_set_1d,dew_point_temperature_set_1d,altimeter_set_1d,sea_level_pressure_set_1d,wet_bulb_temp_set_1d,rain_lbnl_mm,rain_openmeteo_mm
0,0.0,214094.0,2021-04-30 21:50:00+00:00,0.0,-3.5,24.5,footbridge,14514.0,986.4019,13.13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,214095.0,2021-04-30 21:55:00+00:00,0.0,-3.3,24.5,footbridge,14515.0,986.4019,13.14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.0,214096.0,2021-04-30 22:00:00+00:00,0.0,-3.7,24.5,footbridge,14515.0,986.4019,13.14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3.0,24232.0,2021-05-01 00:35:00+00:00,26.5,0.0,10.5,footbridge,14525.0,985.8633,13.16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4.0,24233.0,2021-05-01 01:00:00+00:00,26.5,0.0,10.2,footbridge,14527.0,985.6047,13.16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
514553,NaN,NaN,2025-10-19 00:00:00+00:00,NaN,NaN,NaN,south_fork_2,NaN,NaN,NaN,...,0.0,13.36,NaN,NaN,10.52,101552.54,101435.62,15.72,0.0,0.0
514554,NaN,NaN,2025-10-19 00:00:00+00:00,NaN,NaN,NaN,south_fork_0,NaN,NaN,NaN,...,0.0,13.36,NaN,NaN,10.52,101552.54,101435.62,15.72,0.0,0.0
514555,NaN,NaN,2025-10-19 00:00:00+00:00,NaN,NaN,NaN,south_fork_1,NaN,NaN,NaN,...,0.0,13.36,NaN,NaN,10.52,101552.54,101435.62,15.72,0.0,0.0
514556,NaN,NaN,2025-10-19 00:00:00+00:00,NaN,NaN,NaN,university_house,NaN,NaN,NaN,...,0.0,13.36,NaN,NaN,10.52,101552.54,101435.62,15.72,0.0,0.0


In [32]:
df_old=pd.read_csv('final_creek_and_weather_dataset.csv')

In [33]:
df_old

,Unnamed: 0,Unnamed: 0_x,datetime,conductivity,depth,temperature,location,Unnamed: 0_y,BaroPres,Batt_Volt_Avg,...,Temp2m_Avg,VertWind_Avg,VertWind_Max,WindGust_Max,WndDir,WndDir_res,WndSpd,WndSpd_dup,WndSpd_res,rain_mm
0,0,214094,2021-04-30 21:50:00,0.0,-3.500,24.50,footbridge,14514,986.4019,13.13,...,50.36,0.0,0.0,11.680,145.30,146.90,7.394,7.394,7.120,0.0
1,1,214095,2021-04-30 21:55:00,0.0,-3.300,24.50,footbridge,14515,986.4019,13.14,...,50.15,0.0,0.0,13.970,142.30,142.90,9.520,9.520,9.250,0.0
2,2,214096,2021-04-30 22:00:00,0.0,-3.700,24.50,footbridge,14515,986.4019,13.14,...,50.15,0.0,0.0,13.970,142.30,142.90,9.520,9.520,9.250,0.0
3,3,24232,2021-05-01 00:35:00,26.5,0.000,10.50,footbridge,14525,985.8633,13.16,...,49.10,0.0,0.0,12.980,163.30,165.40,5.345,5.345,4.847,0.0
4,4,24233,2021-05-01 01:00:00,26.5,0.000,10.20,footbridge,14527,985.6047,13.16,...,48.94,0.0,0.0,18.090,204.10,206.90,5.464,5.464,4.603,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
479535,479535,182574,2025-09-04 00:00:00,582.8,495.500,18.90,south_fork_2,163777,984.6421,13.68,...,58.79,0.0,0.0,7.557,55.03,54.19,3.911,3.911,3.686,0.0
479536,479536,194043,2025-09-04 00:00:00,338.7,-3105.912,19.20,university_house,163777,984.6421,13.68,...,58.79,0.0,0.0,7.557,55.03,54.19,3.911,3.911,3.686,0.0
479537,479537,204068,2025-09-04 00:00:00,342.8,106.680,17.50,oxford,163777,984.6421,13.68,...,58.79,0.0,0.0,7.557,55.03,54.19,3.911,3.911,3.686,0.0
479538,479538,744,2025-09-04 00:00:00,620.2,22.700,16.80,kingman_hall,163777,984.6421,13.68,...,58.79,0.0,0.0,7.557,55.03,54.19,3.911,3.911,3.686,0.0
